# Introduction

In this notebook, our focus is four-fold:
We'll demonstrate how to connect to OpenAI's GPT-3.5 using our existing connector, facilitating smooth interaction with the model. 

Follow by showcasing effective methods for creating Moonshot's recipe and cookbook, providing structured approaches for utilizing the GPT-3.5 model across various tasks and domains. 
Then we'll run benchmarks leveraging the Moonshot library to assess performance and efficiency, offering insights into the capabilities of our system.

Lastly aside from Benchmarking, Moonshot’s secret sauce Red Teaming function is added to bolster our system's capabilities. 
This function will enable simulation of adversarial attacks or critical analysis, enhancing security measures and solution robustness.

* Create an endpoint
* Create a recipe
* Create a cookbook
* List and run a recipe
* List and run a cookbook
* Start new session 
* Send prompts to end points
* Add Prompt template and context strategy 
* List Session 
* List prompt templates

## Pre-requisite

If you have not create a virtual environment with this notebook, we suggest creating one to avoid any conflicts in the Python libraries. Once you have created the virtual environment, install all the requirements using the following command:

```pip install -r requirements.txt```

## Import and Environment Variables

Import Moonshot library to use in Jupyter notebook

In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import sys, os, json
sys.path.insert(0, '../../')

import asyncio
from moonshot.api import (
    api_create_recipe,
    api_create_recipe_runner,
    api_create_cookbook,
    api_create_cookbook_runner,
    api_create_endpoint,
    api_create_session,
    api_get_session,
    api_get_all_connector_type,
    api_get_all_endpoint,
    api_get_all_cookbook,
    api_get_all_recipe,
    api_get_all_runner,
    api_get_all_session_detail,
    api_get_all_prompt_template_detail,
    api_get_all_context_strategy_name,
    api_get_session_chats_by_session_id,
    api_load_runner,
    api_read_result,
    api_set_environment_variables,
    api_send_prompt,
    api_update_context_strategy,
    api_update_prompt_template,
)

### To enhance the display of the tables, we utilize a Python library - rich ###
from rich.columns import Columns
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

moonshot_path = "data/"

env = {
    "CONNECTORS_ENDPOINTS": os.path.join(moonshot_path, "connectors-endpoints"),
    "COOKBOOKS": os.path.join(moonshot_path, "cookbooks"),
    "DATABASES": os.path.join(moonshot_path, "databases"),
    "DATASETS": os.path.join(moonshot_path, "datasets"),
    "PROMPT_TEMPLATES": os.path.join(moonshot_path, "prompt-templates"),
    "RECIPES": os.path.join(moonshot_path, "recipes"),
    "RESULTS": os.path.join(moonshot_path, "results"),
    "RUNNERS": os.path.join(moonshot_path, "runners"),
}

api_set_environment_variables(env)

# initialise the global console
console = Console()

Unable to retrieve the following environment variables: ['ATTACK_MODULES', 'CONNECTORS', 'CONTEXT_STRATEGY', 'DATABASES_MODULES', 'IO_MODULES', 'METRICS', 'METRICS_CONFIG', 'RECIPES_MODULES', 'REPORTS', 'REPORTS_MODULES', 'SESSIONS', 'STOP_STRATEGIES']. The stock set will be used.


## Results Display Enhancement Functions

These functions aid in enhancing the presentation of results obtained from Moonshot libraries

<a id='prettified_functions'></a>

In [2]:
def list_connector_types(connector_types):
    if connector_types:
        table = Table("No.", "Connector Type")
        for connector_id, connector_type in enumerate(connector_types, 1):
            table.add_section()
            table.add_row(str(connector_id), connector_type)
        console.print(table)
    else:
        console.print("[red]There are no connector types found.[/red]")
        
def list_endpoints(endpoints_list):
    if endpoints_list:
        table = Table(
            "No.",
            "Id",
            "Name",
            "Connector Type",
            "Uri",
            "Token",
            "Max calls per second",
            "Max concurrency",
            "Params",
            "Created Date",
        )
        for endpoint_id, endpoint in enumerate(endpoints_list, 1):
            (
                id,
                name,
                connector_type,
                uri,
                token,
                max_calls_per_second,
                max_concurrency,
                params,
                created_date,
            ) = endpoint.values()
            table.add_section()
            table.add_row(
                str(endpoint_id),
                id,
                name,
                connector_type,
                uri,
                token,
                str(max_calls_per_second),
                str(max_concurrency),
                str(params),
                created_date,
            )
        console.print(table)
    else:
        console.print("[red]There are no endpoints found.[/red]")

def list_recipes(recipes_list):
    if recipes_list:
        table = Table("No.", "Recipe", "Contains")
        for recipe_id, recipe in enumerate(recipes_list, 1):
            (
                id,
                name,
                description,
                tags,
                datasets,
                prompt_templates,
                metrics,
                rec_type,
                attack_strategies,
            ) = recipe.values()
            recipe_info = (
                f"[red]id: {id}[/red]\n\n[blue]{name}[/blue]\n{description}\n\n"
                f"Tags:\n{tags}\n\nType:\n{rec_type}"
            )

            if datasets:
                datasets_info = "[blue]Datasets[/blue]:" + "".join(
                    f"\n{i + 1}. {item}" for i, item in enumerate(datasets)
                )
            else:
                datasets_info = "[blue]Datasets[/blue]: nil"

            if prompt_templates:
                prompt_templates_info = "[blue]Prompt Templates[/blue]:" + "".join(
                    f"\n{i + 1}. {item}" for i, item in enumerate(prompt_templates)
                )
            else:
                prompt_templates_info = "[blue]Prompt Templates[/blue]: nil"

            if metrics:
                metrics_info = "[blue]Metrics[/blue]:" + "".join(
                    f"\n{i + 1}. {item}" for i, item in enumerate(metrics)
                )
            else:
                metrics_info = "[blue]Metrics[/blue]: nil"

            if attack_strategies:
                attack_strategies_info = "[blue]Attack Strategies[/blue]:" + "".join(
                    f"\n{i + 1}. {item}" for i, item in enumerate(attack_strategies)
                )
            else:
                attack_strategies_info = "[blue]Attack Strategies[/blue]: nil"

            contains_info = f"{datasets_info}\n{prompt_templates_info}\n{metrics_info}\n{attack_strategies_info}"
            table.add_section()
            table.add_row(str(recipe_id), recipe_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no recipes found.[/red]")

def list_cookbooks(cookbooks_list):
    if cookbooks_list:
        table = Table("No.", "Cookbook", "Recipes")
        for cookbook_id, cookbook in enumerate(cookbooks_list, 1):
            id, name, description, recipes = cookbook.values()
            cookbook_info = f"[red]id: {id}[/red]\n\n[blue]{name}[/blue]\n{description}"
            recipes_info = "\n".join(
                f"{i + 1}. {item}" for i, item in enumerate(recipes)
            )
            table.add_section()
            table.add_row(str(cookbook_id), cookbook_info, recipes_info)
        console.print(table)
    else:
        console.print("[red]There are no cookbooks found.[/red]")

def show_recipe_results(recipes, endpoints, recipe_results, duration):
    if recipe_results:
        # Display recipe results
        generate_recipe_table(recipes, endpoints, recipe_results)
    else:
        console.print("[red]There are no results.[/red]")

    # Print run stats
    console.print(f"{'='*50}\n[blue]Time taken to run: {duration}s[/blue]\n{'='*50}")

def show_cookbook_results(cookbooks, endpoints, cookbook_results, duration):
    if cookbook_results:
        # Display recipe results
        generate_cookbook_table(cookbooks, endpoints, cookbook_results)
    else:
        console.print("[red]There are no results.[/red]")

    # Print run stats
    console.print(f"{'='*50}\n[blue]Time taken to run: {duration}s[/blue]\n{'='*50}")

def generate_recipe_table(
        recipes: list, endpoints: list, results: dict
    ) -> None:
    table = Table("", "Recipe", *endpoints)
    for recipe_index, recipe in enumerate(recipes, 1):
        # Get recipe result
        recipe_result = {}
        for tmp_result in results["results"]["recipes"]:
            if tmp_result["id"] == recipe:
                recipe_result = tmp_result
                break

        endpoint_results = list()
        for endpoint in endpoints:
            output_results = {}

            # Get endpoint result
            ep_result = {}
            for tmp_result in recipe_result["models"]:
                if tmp_result["id"] == endpoint:
                    ep_result = tmp_result

            for ds in ep_result["datasets"]:
                for pt in ds["prompt_templates"]:
                    output_results[(ds["id"], pt["id"])] = pt["metrics"]

            endpoint_results.append(str(output_results))
        table.add_section()
        table.add_row(str(recipe_index), recipe, *endpoint_results)
    # Display table
    console.print(table)

def generate_cookbook_table(cookbooks, endpoints: list, results: dict) -> None:
    table = Table("", "Cookbook", "Recipe", *endpoints)
    index = 1
    for cookbook in cookbooks:
        # Get cookbook result
        cookbook_result = {}
        for tmp_result in results["results"]["cookbooks"]:
            if tmp_result["id"] == cookbook:
                cookbook_result = tmp_result
                break

        for recipe in cookbook_result["recipes"]:
            endpoint_results = list()
            for endpoint in endpoints:
                output_results = {}

                # Get endpoint result
                ep_result = {}
                for tmp_result in recipe["models"]:
                    if tmp_result["id"] == endpoint:
                        ep_result = tmp_result

                for ds in ep_result["datasets"]:
                    for pt in ds["prompt_templates"]:
                        output_results[(ds["id"], pt["id"])] = pt["metrics"]

                endpoint_results.append(str(output_results))
            table.add_section()
            table.add_row(str(index), cookbook, recipe["id"], *endpoint_results)
            index += 1

    # Display table
    console.print(table)

def list_runs(runs_list):
    if runs_list:
        table = Table("No.", "Run id", "Contains")
        for run_index, run_data in enumerate(runs_list, 1):
            (
                run_id,
                run_name,
                run_type,
                db_file,
                recipes,
                cookbooks,
                endpoints,
                num_of_prompts,
            ) = run_data.values()
            run_info = f"[red]id: {run_id}[/red]\n"

            contains_info = ""
            if recipes:
                contains_info += (
                    "[blue]Recipes:[/blue]"
                    + "".join(f"\n{i + 1}. {item}" for i, item in enumerate(recipes))
                    + "\n\n"
                )
            elif cookbooks:
                contains_info += (
                    "[blue]Cookbooks:[/blue]"
                    + "".join(f"\n{i + 1}. {item}" for i, item in enumerate(cookbooks))
                    + "\n\n"
                )

            contains_info += (
                "[blue]Endpoints:[/blue]"
                + "".join(f"\n{i + 1}. {item}" for i, item in enumerate(endpoints))
                + "\n\n"
            )
            contains_info += f"[blue]Number of Prompts:[/blue]\n{num_of_prompts}\n\n"
            contains_info += f"[blue]Database path:[/blue]\n{db_file}"

            table.add_section()
            table.add_row(str(run_index), run_info, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no runs found.[/red]")

def list_sessions(session_list):
    if session_list:
        table = Table(title="Session List", show_lines=True)
        table.add_column("No.", style="dim", width=6)
        table.add_column("Session ID", justify="center")
        table.add_column("Contains", justify="left")

        for session_index, session_data in enumerate(session_list, 1):
            session_id = session_data.get("session_id", "")
            name = session_data.get("name", "")
            description = session_data.get("description", "")
            endpoints = ", ".join(session_data.get("endpoints", []))
            created_datetime = session_data.get("created_datetime", "")
            chat_ids = ", ".join(map(str, session_data.get("chat_ids", [])))

            session_info = f"[red]id: {session_id}[/red]\n\nCreated: {created_datetime}"
            contains_info = f"[blue]{name}[/blue]\n{description}\n\n"
            contains_info += f"[blue]Endpoints:[/blue] {endpoints}\n\n"
            contains_info += f"[blue]Chat IDs:[/blue] {chat_ids}"

            table.add_row(str(session_index), session_info, contains_info)
        console.print(Panel(table))
    else:
        console.print("[red]There are no sessions found.[/red]", style="bold")

def list_context_strategy(context_strategies):
    if context_strategies:
        table = Table("No.", "Context Strategies")
        for ct_index, ct_data in enumerate(context_strategies, 1):
            table.add_section()
            table.add_row(str(ct_index), ct_data)
        console.print(table)
    else:
        console.print("[red]There are no context strategies found.[/red]")

def list_prompt_templates(prompt_templates):
    table = Table(
        "No.",
        "Prompt Name",
        "Prompt Description",
        "Prompt Template",
    )
    if prompt_templates:
        for prompt_index, prompt_template in enumerate(prompt_templates, 1):
            (
                prompt_name,
                prompt_description,
                prompt_template_contents,
            ) = prompt_template.values()

            table.add_section()
            table.add_row(
                str(prompt_index),
                prompt_name,
                prompt_description,
                prompt_template_contents,
            )
        console.print(table)
    else:
        console.print("[red]There are no prompt templates found.[/red]")

def show_session_chats(session_chats):
    if session_chats:
        table = Table("No.", "Endpoint", "Contains")
        for chat_index, chat_data in enumerate(session_chats, 1):
            (
                chat_id,
                endpoint,
                chat_history
            ) = chat_data.values()
            for chat_history_index, chat_history_data in enumerate(chat_history, 1):
                (
                    chat_record_id,
                    conn_id,
                    context_strategy,
                    prompt_template,
                    prompt,
                    prepared_prompt,
                    predicted_result,
                    duration,
                    prompt_time
                ) = chat_history_data.values()
                
                contains_info = ""
                contains_info += f"[blue]Chat Record Id:[/blue]\n{chat_record_id}\n\n"
                if conn_id:
                    contains_info += f"[blue]Connection Id:[/blue]\n{conn_id}\n\n"
                else:
                    contains_info += f"[blue]Connection Id:[/blue]\nNone\n\n"

                if context_strategy:
                    contains_info += f"[blue]Context Strategy:[/blue]\n{context_strategy}\n\n"
                else:
                    contains_info += f"[blue]Context Strategy:[/blue]\nNone\n\n"
                
                if prompt_template:
                    contains_info += f"[blue]Prompt Template:[/blue]\n{prompt_template}\n\n"
                else:
                    contains_info += f"[blue]Prompt Template:[/blue]\nNone\n\n"
                    
                contains_info += f"[blue]Prompt[/blue]\n{prompt}\n\n"
                contains_info += f"[blue]Prepared Prompt:[/blue]\n{prepared_prompt}\n\n"
                contains_info += f"[blue]Predicted Result:[/blue]\n{predicted_result}\n\n"
                contains_info += f"[blue]Duration:[/blue]\n{duration}s\n\n"
                contains_info += f"[blue]Prompt Time:[/blue]\n{prompt_time}\n\n"
                table.add_section()
                table.add_row(str(chat_index), endpoint, contains_info)
        console.print(table)
    else:
        console.print("[red]There are no session chats found.[/red]")

def show_session(session_instance):
    if session_instance:
        metadata = session_instance.metadata
        table = Table("Session Id", "Session Info")
        contains_info = ""
        contains_info += f"[blue]Name:[/blue]\n{metadata.name}\n\n"
        contains_info += f"[blue]Description:[/blue]\n{metadata.description}\n\n"
        contains_info += f"[blue]Endpoints:[/blue]\n{metadata.endpoints}\n\n"
        if metadata.context_strategy:
            contains_info += f"[blue]Context Strategy:[/blue]\n{metadata.context_strategy}\n\n"
        else:
            contains_info += f"[blue]Context Strategy:[/blue]\nNone\n\n"
        
        if metadata.prompt_template:
            contains_info += f"[blue]Prompt Template:[/blue]\n{metadata.prompt_template}\n\n"
        else:
            contains_info += f"[blue]Prompt Template:[/blue]\nNone\n\n"

        table.add_section()
        table.add_row(metadata.session_id, contains_info)
        console.print(table)
    else:
        console.print("[red]Session is not found[/red]")

## Create an endpoint

An endpoint in the context of Moonshot refers to the actual configuration used to connect to a model (i.e. connector). Before an endpoint can be created, the `connector` must exist in the list of the connector.

In this section, you will learn how to create an endpoint using an existing connector that we have included in Moonshot.

### Connector Type

We can list the connectors available in Moonshot using `api_get_all_connector_type()` as shown in the cell below. A connector details the following two mandatory behaviors:

1. How to call the model? (For developers, checkout the function `get_response()` in one of the connector python files in `moonshot\data\connectors\`)
   
2. How to process the response return by the model? (For developers, checkout the function `_process_response()`)

In [3]:
connection_types = api_get_all_connector_type()
connection_types

['hf-llama2-13b-gptq',
 'openai-gpt4',
 'claude2',
 'openai-gpt35',
 'openai-gpt35-turbo-16k',
 'hf-gpt2']

#### Enhance presentation of results

The output generated by the Moonshot library can be aesthetically improved using the `rich` library. We have included these enhancement functions for this purpose [cell](#prettified_functions).

In [4]:
list_connector_types(connection_types)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Connector Type         ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ hf-llama2-13b-gptq     │
├─────┼────────────────────────┤
│ 2   │ openai-gpt4            │
├─────┼────────────────────────┤
│ 3   │ claude2                │
├─────┼────────────────────────┤
│ 4   │ openai-gpt35           │
├─────┼────────────────────────┤
│ 5   │ openai-gpt35-turbo-16k │
├─────┼────────────────────────┤
│ 6   │ hf-gpt2                │
└─────┴────────────────────────┘

### Endpoint

In this notebook, we will evaluate `openai-gpt35`. To connect to a model, we need to create an endpoint to the model.

To create a new endpoint, we can use `api_create_endpoint()`.

Once an endpoint has been added to Moonshot, we can use this endpoint to evaluate the model later when we run our benchmark.

Alternatively, you can use it to start red teaming as well, refer to [cell](#red_teaming) to start the red team process

In [5]:
endpoints_list = api_get_all_endpoint()
list_endpoints(endpoints_list)

There are no endpoints found.

In [23]:
api_create_endpoint(
    "test-openai-endpoint", # name: give it a name to retrieve it later
    "openai-gpt35", # connector_type: the model that we want to evaluate
    "", # uri: not required as we use OpenAI library to connect to their models.
    "ADD_NEW_TOKEN_HERE", # token: access token
    10, # max_calls_per_second: the number of max calls per second
    2, # max_concurrency: the number of concurrent call at any one time,
    {
        "temperature": 0
    } # params: any additional required for this model
)

# Refresh
endpoints_list = api_get_all_endpoint()
list_endpoints(endpoints_list)

┏━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃     ┃            ┃            ┃ Connector  ┃     ┃            ┃ Max calls  ┃ Max       ┃            ┃ Created   ┃
┃ No. ┃ Id         ┃ Name       ┃ Type       ┃ Uri ┃ Token      ┃ per second ┃ concurre… ┃ Params     ┃ Date      ┃
┡━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ 1   │ test-open… │ test-open… │ openai-gp… │     │ ADD_NEW_T… │ 10         │ 2         │ {'tempera… │ 2024-04-… │
│     │            │            │            │     │            │            │           │ 0}         │ 21:26:52  │
└─────┴────────────┴────────────┴────────────┴─────┴────────────┴────────────┴───────────┴────────────┴───────────┘

# Create a recipe

A recipe contains all the details required to run a benchmark. It gives Moonshot step-by-step instructions on what to 
do with those details to run a successful benchmark on the selected model.

The recipe includes the following important details:

1. Name of the recipe (to be used later)
2. Dataset
3. Metric(s)
4. Prompt template (s) (if any)

In this notebook, we will create a test dataset to add to our new recipe. All datasets can be found in `moonshot\data\datasets`. 

In [7]:
test_dataset = {
    "name": "test-dataset",
    "description": "This dataset contains questions on general items and its category.",
    "keywords": [
        "general"
    ],
    "categories": [
        "capability"
    ],
    "examples": [
        {
            "input": "What is an apple?",
            "target": "Fruit"
        },
        {
            "input": "What is a chair?",
            "target": "Furniture"
        },
        {
            "input": "What is a laptop?",
            "target": "Electronic"
        },
        {
            "input": "What is a biscuit?",
            "target": "Food"
        }
        ,
        {
            "input": "What is a pear?",
            "target": "Fruit"
        }
    ]
}

# to change later when notebook is shifted
in_file = "data/datasets/test-dataset.json"
json.dump(test_dataset, open(in_file, "w+"), indent=2)

In this notebook, we create a new prompt template to use with this dataset. When this prompt template is activated, an example prompt will be sent to the model in this form using the dataset above:

```
Answer this question:
What is an apple?
A:
```

In [8]:
prompt_template = {
    "name": "Simple Question Answering Template",
    "description": "This is a simple question and answering template.",
    "template": "Answer this question:\n{{ prompt }}\nA:"
}

in_file = "data/prompt-templates/test-prompt-template.json"
json.dump(prompt_template, open(in_file, "w+"), indent=2)

To add a new recipe, we can use `api_create_recipe`. We will use our dataset and prompt template from the previous two cells in this recipe. 

In [9]:
api_create_recipe(
    "Item Category",
    "This recipe is created to test model's ability in answering question.",
    ["tag1"],
    ["test-dataset"],
    ["test-prompt-template"],
    ["exactstrmatch", 'rougescore'],
    "benchmark",
    []
)

recipes_list = api_get_all_recipe()
list_recipes(recipes_list)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Recipe                                                 ┃ Contains                                         ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ id: item-category                                      │ Datasets:                                        │
│     │                                                        │ 1. test-dataset                                  │
│     │ Item Category                                          │ Prompt Templates:                                │
│     │ This recipe is created to test model's ability in      │ 1. test-prompt-template                          │
│     │ answering question.                                    │ Metrics:                                         │
│     │                                                        │ 1. exactstrmatch                                 │
│     │ Tags:                                                  │ 2. rougescore                                    │
│     │ ['tag1']                                               │ Attack Strategies: nil                           │
│     │                                                        │                                                  │
│     │ Type:                                                  │                                                  │
│     │ benchmark                                              │                                                  │
├─────┼────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ 2   │ id: realtime-qa                                        │ Datasets:                                        │
│     │                                                        │ 1. realtimeqa-past                               │
│     │ RealtimeQA                                             │ Prompt Templates: nil                            │
│     │ RealTime QA is a dynamic question answering (QA)       │ Metrics:                                         │
│     │ platform that inquires about the present.              │ 1. exactstrmatch                                 │
│     │                                                        │ Attack Strategies: nil                           │
│     │ Tags:                                                  │                                                  │
│     │ ['safety', 'hallucination']                            │                                                  │
│     │                                                        │                                                  │
│     │ Type:                                                  │                                                  │
│     │ benchmark                                              │                                                  │
├─────┼────────────────────────────────────────────────────────┼──────────────────────────────────────────────────┤
│ 3   │ id: squad-shifts-tnf                                   │ Datasets:                                        │
│     │                                                        │ 1. squad-shifts-tnf                              │
│     │ squad-shifts-tnf                                       │ Prompt Templates: nil                            │
│     │ Zero-shot reading comprehension on paragraphs and      │ Metrics:                                         │
│     │ questions from squadshifts. Augmented to true/false    │ 1. relaxstrmatch                                 │
│     │ statement.                                             │ Attack Strategies: nil                           │
│     │                                                        │                                                  │
│     │ Tags:                                           

# Create a cookbook

A cookbook can encompass multiple recipes, serving to organize and group them together for evaluating a model. 
To add a cookbook, we use `api_create_cookbook`

In [10]:
api_create_cookbook(
    "test-category-cookbook",
    "This cookbook tests if the model is able to group items into different categories",
    ["item-category"]
)

cookbooks_list = api_get_all_cookbook()
list_cookbooks(cookbooks_list)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Cookbook                                                   ┃ Recipes                                      ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ id: test-category-cookbook                                 │ 1. item-category                             │
│     │                                                            │                                              │
│     │ test-category-cookbook                                     │                                              │
│     │ This cookbook tests if the model is able to group items    │                                              │
│     │ into different categories                                  │                                              │
├─────┼────────────────────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ 2   │ id: common-risk-easy                                       │ 1. uciadult-easy                             │
│     │                                                            │ 2. bbq-random                                │
│     │ Easy test sets for Common Risks                            │ 3. winobias                                  │
│     │ This is a cookbook that consists (easy) test sets for      │ 4. challenging-toxicity-prompts-completion   │
│     │ common risks. These test sets are adapted from various     │ 5. realtime-qa                               │
│     │ research and will be expanded in the future.               │ 6. commonsense-morality-easy                 │
│     │                                                            │ 7. jailbreak-dan                             │
│     │                                                            │ 8. advglue                                   │
├─────┼────────────────────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ 3   │ id: common-risk-hard                                       │ 1. uciadult-hard                             │
│     │                                                            │ 2. bbq-random                                │
│     │ Hard test sets for Common Risks                            │ 3. winobias                                  │
│     │ This is a cookbook that consists (hard) test sets for      │ 4. challenging-toxicity-prompts-completion   │
│     │ common risks. These test sets are adapted from various     │ 5. realtime-qa                               │
│     │ research and will be expanded in the future.               │ 6. commonsense-morality-hard                 │
│     │                                                            │ 7. jailbreak-dan                             │
│     │                                                            │ 8. advglue                                   │
├─────┼────────────────────────────────────────────────────────────┼──────────────────────────────────────────────┤
│ 4   │ id: leaderboard-cookbook                                   │ 1. mmlu                                      │
│     │                                                            │ 2. truthfulqa-mcq                            │
│     │ Leaderboard Cookbook                                       │ 3. winogrande                                │
│     │ This cookbook, drawing inspiration from leaderboards like  │ 4. hellaswag                                 │
│     │ HF's OpenLLM and HELM, features popular benchmarks for     │ 5. arc                                       │
│     │ testing model capabilities, with results that may vary     │ 6. gsm8k                                     │
│     │ from actual leaderboard standings.                         │                                              │
├─────┼─────────────────────────────────────────────────

# Run Recipe(s)

We can run multiple recipes on multiple endpoints using `api_create_recipe_runner` as shown below.
- We can use recipe id to identify the recipe in this function.
- The results will be stored in `moonshot/data/results`

In [11]:
recipes = ["item-category", "bbq"]
endpoints = ["test-openai-endpoint"]
num_of_prompts = 5 # use a smaller number to test out the function

rec_runner = api_create_recipe_runner(
    "my new recipe runner",
    recipes,
    endpoints,
    num_of_prompts
)

await rec_runner.run()
rec_runner.close()

# Display results
result_info = api_read_result(rec_runner.id)
show_recipe_results(
    recipes, endpoints, result_info, result_info["metadata"]["duration"]
)

Established connection to database (data/databases/my-new-recipe-runner.db)
[Runner] my-new-recipe-runner - Running...
🔃 Running recipes (['item-category', 'bbq'])... do not close this terminal.
You can start a new terminal to continue working.
[Run] Running recipe item-category... (1/2)
[Run] Part 0: Loading asyncio running loop...
[Run] Part 1: Loading various recipe instances...
[Run] Load recipe instance took 0.0002s


DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is an apple?\\nA:"}], "temperature": 0}' message='Post details'


[Run] Load recipe endpoints instances took 0.8135s
[Run] Load metrics took 0.0099s
[Run] Part 2: Invoke recipe processing module...
[Run] Performing processing for recipe [item-category] using processing module: benchmarking
[Benchmarking] Part 1: Create new cache table if needed...
[Benchmarking] Part 2: Building and executing generator pipeline for predicting prompts...
Predicting prompt 1 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=682 request_id=req_903b3f822815cd7e2b42b2b7d3fd9bc2 response_code=200
DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is a chair?\\nA:"}], "temperature": 0}' message='Post details'


[Prompt 1] took 1.1198s
Predicting prompt 2 [test-openai-endpoint]


...

[Prompt 4] took 1.0560s
Predicting prompt 5 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=520 request_id=req_174b8c9dd053be13c1087bf7da010d8b response_code=200


[Prompt 5] took 1.1552s
[Benchmarking] Predicting prompts for recipe [bbq] took 8.5880s
[Benchmarking] Part 3: Sort the recipe predictions into groups
[Benchmarking] Sort the recipe predictions into groups for recipe [bbq] took 0.0000s
[Benchmarking] Part 4: Performing metrics calculation
[Benchmarking] Running metrics for conn_id (test-openai-endpoint), recipe_id (bbq), dataset_id (bbq-ambiguous), prompt_template_id (bbq-template)
[exactstrmatch] Running [get_results] took 0.0000s
[Benchmarking] Running metrics for conn_id (test-openai-endpoint), recipe_id (bbq), dataset_id (bbq-disamb), prompt_template_id (bbq-template)
[exactstrmatch] Running [get_results] took 0.0000s
[Benchmarking] Performing metrics calculation for recipe [bbq] took 0.0000s
[Run] Performing processing for recipe [bbq] took 8.5905s
[Runner] my-new-recipe-runner - Run completed.
[Runner] my-new-recipe-runner - Writing result...
[Runner] my-new-recipe-runner - Run results written to data/results/my-new-recipe-runner

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Recipe        ┃ test-openai-endpoint                                                                        ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match': 0.0}, {'rouge': {'rouge-1': │
│   │               │ {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': │
│   │               │ 0.0, 'p': 0.0, 'f': 0.0}}}]}                                                                │
├───┼───────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2 │ bbq           │ {('bbq-ambiguous', 'bbq-template'): [{'exact_str_match': 0.6}], ('bbq-disamb',              │
│   │               │ 'bbq-template'): [{'exact_str_match': 1.0}]}                                                │
└───┴───────────────┴─────────────────────────────────────────────────────────────────────────────────────────────┘

==================================================
Time taken to run: 16s
==================================================

# Run a cookbook

To run a cookbook, we can use `api_create_cookbook_runner`. 
- We can run multiple cookbooks on multiple endpoints.
- We can use cookbook id to identify the cookbook in this function.
- The results will be stored in `moonshot/data/results/`

In [12]:
cookbooks = ["test-category-cookbook"]
endpoints = ["test-openai-endpoint"]
num_of_prompts = 1

cb_runner = api_create_cookbook_runner(
    "my new cookbook runner",
    cookbooks,
    endpoints,
    num_of_prompts
)

await cb_runner.run()
cb_runner.close()

# Display results
result_info = api_read_result(cb_runner.id)
show_cookbook_results(
    cookbooks, endpoints, result_info, result_info["metadata"]["duration"]
)

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is an apple?\\nA:"}], "temperature": 0}' message='Post details'


Established connection to database (data/databases/my-new-cookbook-runner.db)
[Runner] my-new-cookbook-runner - Running...
🔃 Running cookbooks (['test-category-cookbook'])... do not close this terminal.
You can start a new terminal to continue working.
[Run] Running cookbook test-category-cookbook... (1/1)
[Run] Part 1: Loading various cookbook instances...
[Run] Load cookbook instance took 0.0005s
[Run] Part 2: Running cookbook recipes...
[Run] Running recipe item-category... (1/1)
[Run] Part 0: Loading asyncio running loop...
[Run] Part 1: Loading various recipe instances...
[Run] Load recipe instance took 0.0006s
[Run] Load recipe endpoints instances took 0.0007s
[Run] Load metrics took 0.0004s
[Run] Part 2: Invoke recipe processing module...
[Run] Performing processing for recipe [item-category] using processing module: benchmarking
[Benchmarking] Part 1: Create new cache table if needed...
[Benchmarking] Part 2: Building and executing generator pipeline for predicting prompts...
P

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=552 request_id=req_4e1bd564bb5e60abc14905a4fac16b1b response_code=200


[Prompt 1] took 1.1085s
[Benchmarking] Predicting prompts for recipe [item-category] took 1.1121s
[Benchmarking] Part 3: Sort the recipe predictions into groups
[Benchmarking] Sort the recipe predictions into groups for recipe [item-category] took 0.0000s
[Benchmarking] Part 4: Performing metrics calculation
[Benchmarking] Running metrics for conn_id (test-openai-endpoint), recipe_id (item-category), dataset_id (test-dataset), prompt_template_id (test-prompt-template)
[exactstrmatch] Running [get_results] took 0.0000s
[rougescore] Running [get_results] took 0.0001s
[Benchmarking] Performing metrics calculation for recipe [item-category] took 0.0001s
[Run] Performing processing for recipe [item-category] took 1.1146s
[Run] Running cookbook [test-category-cookbook] took 1.1183s
[Runner] my-new-cookbook-runner - Run completed.
[Runner] my-new-cookbook-runner - Writing result...
[Runner] my-new-cookbook-runner - Run results written to data/results/my-new-cookbook-runner.json
Closed connect

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Cookbook               ┃ Recipe        ┃ test-openai-endpoint                                               ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ test-category-cookbook │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match':    │
│   │                        │               │ 0.0}, {'rouge': {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},        │
│   │                        │               │ 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0,   │
│   │                        │               │ 'p': 0.0, 'f': 0.0}}}]}                                            │
└───┴────────────────────────┴───────────────┴────────────────────────────────────────────────────────────────────┘

==================================================
Time taken to run: 1s
==================================================

# List all runs

All runs are stored in Moonshot, and you can retrieve your historical runs by using the `api_get_all_runner` function.

Runs prove to be highly beneficial in various scenarios, such as:

1. In the event of a network interruption leading to a halted run midway..
2. When you need to rerun a specific run due to updates made to your model at the same endpoint.

In [13]:
runner_list = api_get_all_runner()
list_runs(runner_list)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Run id                     ┃ Contains                                 ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ id: my-new-recipe-runner   │ Recipes:                                 │
│     │                            │ 1. item-category                         │
│     │                            │ 2. bbq                                   │
│     │                            │                                          │
│     │                            │ Endpoints:                               │
│     │                            │ 1. test-openai-endpoint                  │
│     │                            │                                          │
│     │                            │ Number of Prompts:                       │
│     │                            │ 5                                        │
│     │                            │                                          │
│     │                            │ Database path:                           │
│     │                            │ data/databases/my-new-recipe-runner.db   │
├─────┼────────────────────────────┼──────────────────────────────────────────┤
│ 2   │ id: my-new-cookbook-runner │ Cookbooks:                               │
│     │                            │ 1. test-category-cookbook                │
│     │                            │                                          │
│     │                            │ Endpoints:                               │
│     │                            │ 1. test-openai-endpoint                  │
│     │                            │                                          │
│     │                            │ Number of Prompts:                       │
│     │                            │ 1                                        │
│     │                            │                                          │
│     │                            │ Database path:                           │
│     │                            │ data/databases/my-new-cookbook-runner.db │
└─────┴────────────────────────────┴──────────────────────────────────────────┘

## Resume a run

To resume a run, you can use `api_load_runner`.

In [14]:
# Resume a recipe run
run_id = "my-new-recipe-runner" # replace this with one of the run IDs shown above
rec_runner = api_load_runner(run_id)
await rec_runner.run()
rec_runner.close()

# Display results
result_info = api_read_result(rec_runner.id)
show_recipe_results(
    recipes, endpoints, result_info, result_info["metadata"]["duration"]
)

Established connection to database (data/databases/my-new-recipe-runner.db)
[Runner] my-new-recipe-runner - Running...
🔃 Running recipes (['item-category', 'bbq'])... do not close this terminal.
You can start a new terminal to continue working.
[Run] Running recipe item-category... (1/2)
[Run] Part 0: Loading asyncio running loop...
[Run] Part 1: Loading various recipe instances...
[Run] Load recipe instance took 0.0003s
[Run] Load recipe endpoints instances took 0.0005s
[Run] Load metrics took 0.0003s
[Run] Part 2: Invoke recipe processing module...
[Run] Performing processing for recipe [item-category] using processing module: benchmarking
[Benchmarking] Part 1: Create new cache table if needed...
[Benchmarking] Part 2: Building and executing generator pipeline for predicting prompts...
[Benchmarking] Predicting prompts for recipe [item-category] took 0.0027s
[Benchmarking] Part 3: Sort the recipe predictions into groups
[Benchmarking] Sort the recipe predictions into groups for reci

┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Recipe        ┃ test-openai-endpoint                                                                        ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match': 0.0}, {'rouge': {'rouge-1': │
│   │               │ {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': │
│   │               │ 0.0, 'p': 0.0, 'f': 0.0}}}]}                                                                │
├───┼───────────────┼─────────────────────────────────────────────────────────────────────────────────────────────┤
│ 2 │ bbq           │ {('bbq-ambiguous', 'bbq-template'): [{'exact_str_match': 0.6}], ('bbq-disamb',              │
│   │               │ 'bbq-template'): [{'exact_str_match': 1.0}]}                                                │
└───┴───────────────┴─────────────────────────────────────────────────────────────────────────────────────────────┘

==================================================
Time taken to run: 0s
==================================================

In [15]:
# Resume a cookbook run
run_id = "my-new-cookbook-runner" # replace this with one of the run IDs shown above
cb_runner = api_load_runner(run_id)
await cb_runner.run()
cb_runner.close()

# Display results
result_info = api_read_result(cb_runner.id)
show_cookbook_results(
    cookbooks, endpoints, result_info, result_info["metadata"]["duration"]
)

Established connection to database (data/databases/my-new-cookbook-runner.db)
[Runner] my-new-cookbook-runner - Running...
🔃 Running cookbooks (['test-category-cookbook'])... do not close this terminal.
You can start a new terminal to continue working.
[Run] Running cookbook test-category-cookbook... (1/1)
[Run] Part 1: Loading various cookbook instances...
[Run] Load cookbook instance took 0.0003s
[Run] Part 2: Running cookbook recipes...
[Run] Running recipe item-category... (1/1)
[Run] Part 0: Loading asyncio running loop...
[Run] Part 1: Loading various recipe instances...
[Run] Load recipe instance took 0.0003s
[Run] Load recipe endpoints instances took 0.0005s
[Run] Load metrics took 0.0003s
[Run] Part 2: Invoke recipe processing module...
[Run] Performing processing for recipe [item-category] using processing module: benchmarking
[Benchmarking] Part 1: Create new cache table if needed...
[Benchmarking] Part 2: Building and executing generator pipeline for predicting prompts...
[

┏━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Cookbook               ┃ Recipe        ┃ test-openai-endpoint                                               ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1 │ test-category-cookbook │ item-category │ {('test-dataset', 'test-prompt-template'): [{'exact_str_match':    │
│   │                        │               │ 0.0}, {'rouge': {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},        │
│   │                        │               │ 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.0,   │
│   │                        │               │ 'p': 0.0, 'f': 0.0}}}]}                                            │
└───┴────────────────────────┴───────────────┴────────────────────────────────────────────────────────────────────┘

==================================================
Time taken to run: 0s
==================================================

## Red Teaming <a id='red_teaming'></a>

### Create a Red Teaming session

In moonshot, you are able to start a red team session with 1 or more end points. To start, give it a name description and the end point(s).

In [16]:
endpoints = ["test-openai-endpoint"]

my_rt_session = api_create_session(
    "My Red Teaming Session",
    "Creating a new red teaming description",
    endpoints,
)

session_id = my_rt_session.metadata.session_id
show_session(my_rt_session)

Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)
Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Session Id                             ┃ Session Info                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ my-red-teaming-session_20240410-212619 │ Name:                                  │
│                                        │ My Red Teaming Session                 │
│                                        │                                        │
│                                        │ Description:                           │
│                                        │ Creating a new red teaming description │
│                                        │                                        │
│                                        │ Endpoints:                             │
│                                        │ ['test-openai-endpoint']               │
│                                        │                                        │
│                                        │ Context Strategy:                      │
│                                        │ None                                   │
│                                        │                                        │
│                                        │ Prompt Template:                       │
│                                        │ None                                   │
│                                        │                                        │
│                                        │                                        │
└────────────────────────────────────────┴────────────────────────────────────────┘

## Send prompt to the endpoints

Once the session with the selected endpoint(s) is established, you can now type the prompt that you would like to send to the model(s) to test. 

In [17]:
prompt = "What is the largest fruit"

await api_send_prompt(session_id, prompt)

show_session_chats(api_get_session_chats_by_session_id(session_id))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "What is the largest fruit"}], "temperature": 0}' message='Post details'


Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)
Predicting prompt 1 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=417 request_id=req_b4ae1088f2a96427fe8fe9d869a885c9 response_code=200


[Prompt 1] took 0.7698s
Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Endpoint             ┃ Contains                                                                           ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ test-openai-endpoint │ Chat Record Id:                                                                    │
│     │                      │ 1                                                                                  │
│     │                      │                                                                                    │
│     │                      │ Connection Id:                                                                     │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Context Strategy:                                                                  │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt Template:                                                                   │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt                                                                             │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Prepared Prompt:                                                                   │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Predicted Result:                                                                  │
│     │                      │ The largest fruit in the world is the jackfruit, which can weigh up to 80 pounds   │
│     │                      │ and grow up to 3 feet in length.                                                   │
│     │                      │                                                                                    │
│     │                      │ Duration:                                                                          │
│     │                      │ 0.769788791978499s                                                                 │
│     │                      │                                                                                    │
│     │                      │ Prompt Time:                                                                       │
│     │                      │ 04/10/2024, 21:26:19                                                               │
│     │                      │                                                                                    │
│     │                      │                                                                                    │
└─────┴──────────────────────┴────────────────────────────────────────────────────────────────────────────────────┘

## Set Context Strategy and Prompt Template.

By indicating Context Strategy, you will be including n-number of previous prompts context to be included in your prompt

Prompt Template serves as a skeleton for constructing input text that prompts the model to generate.


In [18]:
context_strategy = "add_previous_prompt"
prompt_template = "test-prompt-template"

api_update_context_strategy(session_id, context_strategy)
api_update_prompt_template(session_id, prompt_template)

# Get updated session
updated_session = api_get_session(session_id)
show_session(updated_session)

Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)
Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)
Established connection to database (data/databases/my-red-teaming-session_20240410-212619.db)
Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Session Id                             ┃ Session Info                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ my-red-teaming-session_20240410-212619 │ Name:                                  │
│                                        │ My Red Teaming Session                 │
│                                        │                                        │
│                                        │ Description:                           │
│                                        │ Creating a new red teaming description │
│                                        │                                        │
│                                        │ Endpoints:                             │
│                                        │ ['test-openai-endpoint']               │
│                                        │                                        │
│                                        │ Context Strategy:                      │
│                                        │ add_previous_prompt                    │
│                                        │                                        │
│                                        │ Prompt Template:                       │
│                                        │ test-prompt-template                   │
│                                        │                                        │
│                                        │                                        │
└────────────────────────────────────────┴────────────────────────────────────────┘

In [19]:
prompt = "What is the largest animal"

await api_send_prompt(session_id, prompt)

show_session_chats(api_get_session_chats_by_session_id(session_id))

DEBUG:openai:message='Request to OpenAI API' method=post path=https://api.openai.com/v1/chat/completions
DEBUG:openai:api_version=None data='{"model": "gpt-3.5-turbo", "messages": [{"role": "user", "content": "Answer this question:\\nWhat is the largest animalWhat is the largest fruit\\n\\nA:"}], "temperature": 0}' message='Post details'


Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)
Predicting prompt 1 [test-openai-endpoint]


INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=501 request_id=req_e33ea6e07bbf57b0df69df12df9fc9ca response_code=200


[Prompt 1] took 0.8299s
Established connection to database (/Users/lionelteo/Documents/moonshot/examples/jupyter-notebook/../../moonshot/data/sessions/my-red-teaming-session_20240410-212619.db)


┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Endpoint             ┃ Contains                                                                           ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ test-openai-endpoint │ Chat Record Id:                                                                    │
│     │                      │ 1                                                                                  │
│     │                      │                                                                                    │
│     │                      │ Connection Id:                                                                     │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Context Strategy:                                                                  │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt Template:                                                                   │
│     │                      │ None                                                                               │
│     │                      │                                                                                    │
│     │                      │ Prompt                                                                             │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Prepared Prompt:                                                                   │
│     │                      │ What is the largest fruit                                                          │
│     │                      │                                                                                    │
│     │                      │ Predicted Result:                                                                  │
│     │                      │ The largest fruit in the world is the jackfruit, which can weigh up to 80 pounds   │
│     │                      │ and grow up to 3 feet in length.                                                   │
│     │                      │                                                                                    │
│     │                      │ Duration:                                                                          │
│     │                      │ 0.769788791978499s                                                                 │
│     │                      │                                                                                    │
│     │                      │ Prompt Time:                                                                       │
│     │                      │ 04/10/2024, 21:26:19                                                               │
│     │                      │                                                                                    │
│     │                      │                                                                                    │
├─────┼──────────────────────┼────────────────────────────────────────────────────────────────────────────────────┤
│ 1   │ test-openai-endpoint │ Chat Record Id:                                                                    │
│     │                      │ 2                        

## List all Context Strategies

To view all the context strategies that you have created, use the following:

In [20]:
context_strategies = api_get_all_context_strategy_name()
list_context_strategy(context_strategies)

There are no context strategies found.

## List all Prompt Templates

We presented a systematic approach for you to list available prompt templates, you will be able to retrieve the name, description and the template context by calling the following function

In [21]:
prompt_templates = api_get_all_prompt_template_detail()
list_prompt_templates(prompt_templates)

┏━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ No. ┃ Prompt Name                        ┃ Prompt Description                           ┃ Prompt Template       ┃
┡━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ 1   │ Simple Question Answering Template │ This is a simple question and answering      │ Answer this question: │
│     │                                    │ template.                                    │ {{ prompt }}          │
│     │                                    │                                              │ A:                    │
└─────┴────────────────────────────────────┴──────────────────────────────────────────────┴───────────────────────┘

## List all session names

To view all past session, users can call the list all session functions to view the ID, name, description and time stamp of the session created. 

Along with the context strategy and prompt templates used.


In [22]:
sessions = api_get_all_session_detail()
list_sessions(sessions)

There are no sessions found.